In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [ ]:
train_path='C:/Users/acer/Desktop/trina/train'
pred_path='C:/Users/acer/Desktop/trina/prediction'

In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()
        
      
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)      
        self.bn1=nn.BatchNorm2d(num_features=12)    
        self.relu1=nn.ReLU()
      
        self.pool=nn.MaxPool2d(kernel_size=2)

        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1) 
        self.relu2=nn.ReLU()
         
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.bn3=nn.BatchNorm2d(num_features=32)
        self.relu3=nn.ReLU()
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            

In [ ]:
checkpoint=torch.load('best_checkpoint_relu.model')
model=ConvNet(num_classes=10)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=10, bias=True)
)

In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

In [ ]:
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred
    


In [ ]:
images_path=glob.glob(pred_path+'/*.jpeg')

In [ ]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [ ]:
pred_dict

{'prediction\\1.JPEG': 'bucket',
 'prediction\\10.JPEG': 'tower',
 'prediction\\12.JPEG': 'orange',
 'prediction\\13.JPEG': 'bucket',
 'prediction\\14.JPEG': 'elephant',
 'prediction\\15.JPEG': 'elephant',
 'prediction\\16.JPEG': 'elephant',
 'prediction\\17.JPEG': 'banana',
 'prediction\\18.JPEG': 'banana',
 'prediction\\19.JPEG': 'orange',
 'prediction\\2.JPEG': 'orange',
 'prediction\\20.JPEG': 'elephant',
 'prediction\\21.JPEG': 'keyboard',
 'prediction\\22.JPEG': 'schoolbus',
 'prediction\\23.JPEG': 'bucket',
 'prediction\\24.JPEG': 'banana',
 'prediction\\25.JPEG': 'elephant',
 'prediction\\26.JPEG': 'schoolbus',
 'prediction\\27.JPEG': 'orange',
 'prediction\\28.JPEG': 'pizza',
 'prediction\\29.JPEG': 'schoolbus',
 'prediction\\3.JPEG': 'banana',
 'prediction\\30.JPEG': 'banana',
 'prediction\\31.JPEG': 'elephant',
 'prediction\\32.JPEG': 'banana',
 'prediction\\33.JPEG': 'banana',
 'prediction\\34.JPEG': 'pizza',
 'prediction\\35.JPEG': 'elephant',
 'prediction\\36.JPEG': 'elep